Problem 1

In [1]:
%reset -f
from functions import *

l = sp.Integer(200)
h = sp.Integer(400)

theta = 60
d = h
a = sp.sqrt(l**2 - (l/2)**2)
alpha = -90

display(theta, d, a, alpha)

T = transformation_from_denavit_hartenberg([[theta, d, a, alpha]], deg=True)
display(T)
display(T.evalf(3))

60

400

100*sqrt(3)

-90

Matrix([
[      1/2,  0, -sqrt(3)/2, 50*sqrt(3)],
[sqrt(3)/2,  0,        1/2,        150],
[        0, -1,          0,        400],
[        0,  0,          0,          1]])

Matrix([
[  0.5,    0, -0.866,  86.6],
[0.866,    0,    0.5, 150.0],
[    0, -1.0,      0, 400.0],
[    0,    0,      0,   1.0]])

Problem 2

In [11]:
%reset -f
from functions import *

t = sp.symbols('t', real = True, positive = True)
theta1, d2, theta3, theta4 = [sp.Function(name)(t) for name in ['theta1', 'd2', 'theta3', 'theta4']]

DH_list = [[theta1,100,0,pi/2],[pi/2,d2,0,pi/2],[theta3,0,60,pi/2],[theta4,0,0,pi/2]]
print('Denavit-Hartenberg parameters:')
display(sp.Matrix(DH_list))

values = {theta1:135/180*pi, d2:200, theta3:pi/4, theta4:-pi/3}

T = transformation_from_denavit_hartenberg(DH_list)
print(f"Transformation matrix with: theta1, d2, theta3, theta4 = {list(values.values())}")
display(T.evalf(4,subs=values))

Jacobian = Jacobian_from_denavit_hartenberg(DH_list)
print('Jacobian matrix:')
display(Jacobian.evalf(4,subs=values))

q_dot = sp.Matrix([0.0025*pi , 0 , 0 , 0])
v_omega = Jacobian*q_dot
print('Linear and angular velocity:')
display(v_omega.evalf(4,subs=values))

Denavit-Hartenberg parameters:


Matrix([
[theta1(t),   100,  0, pi/2],
[     pi/2, d2(t),  0, pi/2],
[theta3(t),     0, 60, pi/2],
[theta4(t),     0,  0, pi/2]])

Transformation matrix with: theta1, d2, theta3, theta4 = [0.75*pi, 200, pi/4, -pi/3]


Matrix([
[ 0.8624,   -0.5, -0.07946, 171.4],
[-0.3624,   -0.5,  -0.7866, 171.4],
[ 0.3536, 0.7071,  -0.6124, 142.4],
[      0,      0,        0,   1.0]])

Jacobian matrix:


Matrix([
[-171.4,   30.0,    30.0,      0],
[ 171.4,  -30.0,    30.0,      0],
[     0,      0,  -42.43,      0],
[     0, 0.7071, -0.7071,   -0.5],
[     0, 0.7071,  0.7071,   -0.5],
[   1.0,      0,       0, 0.7071]])

Linear and angular velocity:


Matrix([
[  -1.346],
[   1.346],
[       0],
[       0],
[       0],
[0.007854]])